<a href="https://colab.research.google.com/github/Megancodes2017/Megancodes2017/blob/main/Schwarber_pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pybaseball

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 43.7 MB/s eta 0:00:00


In [ ]:
from pybaseball import statcast_batter, batting_stats
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

In [ ]:
# Load in data for all hitters from 2015-2024
years = list(range(2015, 2025))
all_stats = pd.concat([batting_stats(year) for year in years], ignore_index=True)
all_stats = all_stats[all_stats['PA'] >= 100]

In [ ]:
# Create future target (next year's wOBA)
all_stats['next_year'] = all_stats['Season'] + 1
future = all_stats[['Name', 'Season', 'wOBA']].rename(columns={'wOBA': 'next_year_woba', 'Season': 'next_year'})
merged = all_stats.merge(future, on=['Name', 'next_year'])

In [ ]:
# Feature selection and engineering
features = ['Season', 'Age', 'PA', 'BB%', 'K%', 'AVG', 'OBP', 'SLG', 'ISO', 'BABIP', 'wRC+', 'WAR']
merged = merged.dropna(subset=features + ['next_year_woba'])

X = merged[features]
y = merged['next_year_woba']

In [ ]:
# Train test split and model fitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model= LGBMRegressor()
model.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000260 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1261
[LightGBM] [Info] Number of data points in the train set: 621, number of used features: 12
[LightGBM] [Info] Start training from score 0.342498
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

LGBMRegressor()

In [ ]:
# Evaluate
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'MSE Test: {mse:.4f}')

MSE Test: 0.0010


In [ ]:
# Predict for Kyle Schwarber
kyle = all_stats[all_stats['Name'] == 'Kyle Schwarber'].sort_values(by='Season', ascending=False).head(1)

if not kyle.empty:
  kyle_input = kyle[features]
  kyle_woba_pred = model.predict(kyle_input)
  print(f'Predicted next year wOBA for Kyle Schwarber: {kyle_woba_pred[0]:.3f}')
else:
  print('Kyle Schwarber data not found')

Predicted next year wOBA for Kyle Schwarber: 0.347


In [ ]:
# Create multi year future targets
for i in range(1, 4):
  all_stats[f'future_year_{i}'] = all_stats['Season'] + i

future_targets = []
for i in range(1, 4):
  future = all_stats[['Name', f'future_year_{i}', 'wOBA']].rename(
      columns = {'wOBA': f'wOBA_plus_{i}', f'future_year_{i}': 'Season'})
  future_targets.append(future)

merged = all_stats.copy()
for i in range(1, 4):
  merged = merged.merge(future_targets[i-1], on=['Name', 'Season'], how='left')

In [ ]:
# Feature selection and engineering
features = ['Season', 'Age', 'PA', 'BB%', 'K%', 'AVG', 'OBP', 'SLG', 'ISO', 'BABIP', 'wRC+', 'WAR']
targets = ['wOBA_plus_1', 'wOBA_plus_2', 'wOBA_plus_3']

merged = merged.dropna(subset=features + targets)

X = merged[features]

In [ ]:
# Train one model per future year
models = {}
predictions = {}

for i, target in enumerate(targets, start=1):
  y = merged[target]
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
  model = LGBMRegressor()
  model.fit(X_train, y_train)
  models[target] = model
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  print(f'MSE Test for {target}: {mse:.4f}')
  predictions[target] = y_pred


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000080 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 617
[LightGBM] [Info] Number of data points in the train set: 220, number of used features: 12
[LightGBM] [Info] Start training from score 0.349432
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be

In [ ]:
# Predict for Kyle Schwarber next 3 seasons
kyle = all_stats[all_stats['Name'] == 'Kyle Schwarber'].sort_values(by='Season', ascending=False).head(1)

if not kyle.empty:
  kyle_input = kyle[features]
  for target in targets:  # Iterate through the target names
    kyle_pred = models[target].predict(kyle_input)
    # Extract the year number from the target string for printing
    year_number = target.split('_')[-1]
    print(f'Predicted wOBA for Kyle Schwarber in +{year_number}: {kyle_pred[0]:.3f}')
else:
  print('Kyle Schwarber data not found')

Predicted wOBA for Kyle Schwarber in +1: 0.363
Predicted wOBA for Kyle Schwarber in +2: 0.369
Predicted wOBA for Kyle Schwarber in +3: 0.386
